In [1]:
# https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
# Read Parquet File
# !pip install pandas pyarrow
# !pip install pandas fastparquet
import pandas as pd

# df = pd.read_parquet("Yellow_Taxi_Trip_Records_2025.parquet", engine="pyarrow")  # or engine="fastparquet"
# print(df.head())

# # Save as CSV file
# df.to_csv("uber_data.csv", index=False)  # index=False prevents writing row numbers

# print("Parquet file successfully converted to CSV!")

## In Pyspark
# pip install pyspark
# from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("ParquetReader").getOrCreate()
# df = spark.read.parquet("Yellow_Taxi_Trip_Records_2025.parquet")
# df.show()



In [2]:
import pandas as pd
df = pd.read_csv("uber_data.csv", low_memory=False)   #DtypeWarning: Columns (6) have mixed types

# , low_memory=False --> This tells pandas to process the file in larger chunks, reducing memory efficiency but handling mixed types better.
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,7.2,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,5.8,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0


In [3]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3475226 entries, 0 to 3475225
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           int64  
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
 18  Airport_fee            float64
dtypes: float64(12), int64(4), object(3)
memory usage: 503.8+ MB


In [4]:
#Convert the flat file into Data Model Structure
# convert the object into datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3475226 entries, 0 to 3475225
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [5]:
# Drop Duplicates - This will give non duplicate values
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)



In [6]:
datetime_dim

,tpep_pickup_datetime,tpep_dropoff_datetime
0,2025-01-01 00:18:38,2025-01-01 00:26:59
1,2025-01-01 00:32:40,2025-01-01 00:35:13
2,2025-01-01 00:44:04,2025-01-01 00:46:01
3,2025-01-01 00:14:27,2025-01-01 00:20:01
4,2025-01-01 00:21:34,2025-01-01 00:25:06
...,...,...
3412011,2025-01-31 23:01:48,2025-01-31 23:16:29
3412012,2025-01-31 23:50:29,2025-02-01 00:17:27
3412013,2025-01-31 23:26:59,2025-01-31 23:43:01
3412014,2025-01-31 23:14:34,2025-01-31 23:34:52


In [7]:
print(datetime_dim.columns)

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], dtype='object')


In [11]:
#Extract the needed values like hour, day, month, year...
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday


datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
datetime_dim



,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,2025-01-01 00:18:38,2025-01-01 00:26:59,0,1,1,2025,2,0,1,1,2025,2
1,2025-01-01 00:32:40,2025-01-01 00:35:13,0,1,1,2025,2,0,1,1,2025,2
2,2025-01-01 00:44:04,2025-01-01 00:46:01,0,1,1,2025,2,0,1,1,2025,2
3,2025-01-01 00:14:27,2025-01-01 00:20:01,0,1,1,2025,2,0,1,1,2025,2
4,2025-01-01 00:21:34,2025-01-01 00:25:06,0,1,1,2025,2,0,1,1,2025,2
...,...,...,...,...,...,...,...,...,...,...,...,...
3412011,2025-01-31 23:01:48,2025-01-31 23:16:29,23,31,1,2025,4,23,31,1,2025,4
3412012,2025-01-31 23:50:29,2025-02-01 00:17:27,23,31,1,2025,4,0,1,2,2025,5
3412013,2025-01-31 23:26:59,2025-01-31 23:43:01,23,31,1,2025,4,23,31,1,2025,4
3412014,2025-01-31 23:14:34,2025-01-31 23:34:52,23,31,1,2025,4,23,31,1,2025,4


In [13]:
# Assign the index
datetime_dim['datetime_id'] = datetime_dim.index # primary key for the datetime dimension table


In [16]:
datetime_dim

,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_hour,drop_day,drop_month,drop_year,drop_weekday,datetime_id
0,2025-01-01 00:18:38,2025-01-01 00:26:59,0,1,1,2025,2,0,1,1,2025,2,0
1,2025-01-01 00:32:40,2025-01-01 00:35:13,0,1,1,2025,2,0,1,1,2025,2,1
2,2025-01-01 00:44:04,2025-01-01 00:46:01,0,1,1,2025,2,0,1,1,2025,2,2
3,2025-01-01 00:14:27,2025-01-01 00:20:01,0,1,1,2025,2,0,1,1,2025,2,3
4,2025-01-01 00:21:34,2025-01-01 00:25:06,0,1,1,2025,2,0,1,1,2025,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412011,2025-01-31 23:01:48,2025-01-31 23:16:29,23,31,1,2025,4,23,31,1,2025,4,3412011
3412012,2025-01-31 23:50:29,2025-02-01 00:17:27,23,31,1,2025,4,0,1,2,2025,5,3412012
3412013,2025-01-31 23:26:59,2025-01-31 23:43:01,23,31,1,2025,4,23,31,1,2025,4,3412013
3412014,2025-01-31 23:14:34,2025-01-31 23:34:52,23,31,1,2025,4,23,31,1,2025,4,3412014


In [20]:
datetime_dim = datetime_dim[['datetime_id','tpep_pickup_datetime','pick_hour','pick_day','pick_month','pick_year','pick_weekday','tpep_dropoff_datetime','drop_hour','drop_day','drop_month','drop_year','drop_weekday']]


In [21]:
datetime_dim # table created

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2025-01-01 00:18:38,0,1,1,2025,2,2025-01-01 00:26:59,0,1,1,2025,2
1,1,2025-01-01 00:32:40,0,1,1,2025,2,2025-01-01 00:35:13,0,1,1,2025,2
2,2,2025-01-01 00:44:04,0,1,1,2025,2,2025-01-01 00:46:01,0,1,1,2025,2
3,3,2025-01-01 00:14:27,0,1,1,2025,2,2025-01-01 00:20:01,0,1,1,2025,2
4,4,2025-01-01 00:21:34,0,1,1,2025,2,2025-01-01 00:25:06,0,1,1,2025,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412011,3412011,2025-01-31 23:01:48,23,31,1,2025,4,2025-01-31 23:16:29,23,31,1,2025,4
3412012,3412012,2025-01-31 23:50:29,23,31,1,2025,4,2025-02-01 00:17:27,0,1,2,2025,5
3412013,3412013,2025-01-31 23:26:59,23,31,1,2025,4,2025-01-31 23:43:01,23,31,1,2025,4
3412014,3412014,2025-01-31 23:14:34,23,31,1,2025,4,2025-01-31 23:34:52,23,31,1,2025,4


In [24]:
passenger_count_dim = df[['passenger_count']].drop_duplicates().reset_index(drop=True)

passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

In [25]:
passenger_count_dim

,passenger_count_id,passenger_count
0,0,1.0
1,1,3.0
2,2,2.0
3,3,0.0
4,4,4.0
5,5,9.0
6,6,6.0
7,7,5.0
8,8,8.0
9,9,7.0
